<a href="https://colab.research.google.com/github/felipesayegg/Aluguel/blob/main/KNN_aluguel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Entendimento do problema

Muitos fatores influenciam o valor que você pagará pelo aluguel de um imóvel. É importante entender o que afeta essa variação de preço, tanto para cima quanto para baixo. Abaixo, listamos alguns dos principais fatores que podem impactar os custos de aluguel.

Entre os atributos considerados neste estudo estão:


* Área do imóvel (em metros quadrados)
* Número de quartos
* Número de banheiros
* Vagas na garagem
* Andar do imóvel
* Aceita animais
* Imóvel mobiliado
* Valor do condomínio
* Valor do aluguel
* Valor do IPTU
* Valor do seguro


Esses fatores desempenham papéis importantes na determinação do preço final do aluguel e serão analisados no modelo de machine learning para prever o custo com base nesses atributos.

In [3]:
import pandas as pd # importando a biblioteca pandas

In [6]:
df = pd.read_csv('/content/aluguel.csv', sep=';') # carregando os dados

In [7]:
df.head(10)  # vendo meu data frame os 10 primeiros

,cidade,area,quartos,banheiros,vagas_garagem,andar,aceita_animal,mobiliaria,valor_condominio,valor_aluguel,valor_iptu,valor_seguro,valor_total
0,São Paulo,70,2,1,1,7,1,1,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,1,0,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,1,0,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,1,0,270,1112,22,17,1421
4,São Paulo,25,1,1,0,1,0,0,0,800,25,11,836
5,São Paulo,376,3,3,7,0,1,0,0,8000,834,121,8955
6,Rio de Janeiro,72,2,1,0,7,1,0,740,1900,85,25,2750
7,São Paulo,213,4,4,4,4,1,0,2254,3223,1735,41,7253
8,São Paulo,152,2,2,1,3,1,1,1000,15000,250,191,16440
9,Rio de Janeiro,35,1,1,0,2,1,1,590,2300,35,30,2955


## Linhas do df.describe()

count: Número de entradas não nulas em cada coluna. Isso indica quantos dados válidos você tem em cada coluna.


# mean:

Média aritmética dos valores em cada coluna. É a soma de todos os valores dividida pelo número de entradas.


# std:

Desvio padrão dos valores em cada coluna. Mede a dispersão dos dados em relação à média, indicando o quanto os valores variam.


# min:
 Valor mínimo encontrado em cada coluna. É o menor valor presente nos dados.


##25% (1º Quartil ou Q1):

Definição: O primeiro quartil é o valor que divide o menor 25% dos dados do restante. Em outras palavras, 25% dos dados estão abaixo deste valor.
Interpretação: Se você ordenar todos os dados do menor para o maior, o primeiro quartil é o ponto onde um quarto dos dados são menores. Ele ajuda a identificar o limite inferior dos dados, destacando o início da dispersão dos valores.


##50% (Mediana ou Q2):

Definição: A mediana é o valor central que divide os dados em duas metades iguais. Metade dos dados está abaixo da mediana, e a outra metade está acima.
Interpretação: A mediana é uma medida de tendência central que não é influenciada por valores extremos (outliers), tornando-a uma representação precisa do centro dos dados.





##75% (3º Quartil ou Q3):

Definição: O terceiro quartil é o valor abaixo do qual 75% dos dados caem. Isso significa que apenas 25% dos dados são maiores que este valor.
Interpretação: O terceiro quartil indica o limite superior dos dados, mostrando onde três quartos dos valores se encontram. Ele é útil para entender a dispersão superior e identificar possíveis outliers.

max: Valor máximo encontrado em cada coluna. É o maior valor presente nos dados.

In [8]:
df.describe() # vendo algumas informações estatísticas

,area,quartos,banheiros,vagas_garagem,andar,aceita_animal,mobiliaria,valor_condominio,valor_aluguel,valor_iptu,valor_seguro,valor_total
count,10685.000000,10685.00000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000,10685.000000
mean,149.155452,2.50613,2.235751,1.608797,5.043332,0.777913,0.243706,918.745905,3890.416004,335.563032,53.218811,5198.132616
std,537.159483,1.17086,1.406287,1.588426,5.358497,0.415669,0.429338,1166.977706,3381.827918,677.795945,47.350550,4473.376271
min,11.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,450.000000,0.000000,3.000000,499.000000
25%,56.000000,2.00000,1.000000,0.000000,1.000000,1.000000,0.000000,170.000000,1530.000000,38.000000,21.000000,2061.000000
50%,90.000000,2.00000,2.000000,1.000000,3.000000,1.000000,0.000000,560.000000,2650.000000,125.000000,36.000000,3578.000000
75%,182.000000,3.00000,3.000000,2.000000,8.000000,1.000000,0.000000,1235.000000,5000.000000,375.000000,68.000000,6758.000000
max,46335.000000,13.00000,10.000000,12.000000,51.000000,1.000000,1.000000,32000.000000,30000.000000,28120.000000,451.000000,32750.000000


# 3. Pre- Processamento




In [12]:
x_area_mobiliado_quartos = df[["area", "mobiliaria", "quartos"]]  # colocando a
y_aluguel = df["valor_aluguel"]

In [13]:
x_area_mobiliado_quartos

,area,mobiliaria,quartos
0,70,1,2
1,320,0,4
2,80,0,1
3,51,0,2
4,25,0,1
...,...,...,...
10680,63,1,2
10681,285,0,4
10682,70,1,3
10683,120,1,2


In [14]:
y_aluguel

,valor_aluguel
0,3300
1,4960
2,2800
3,1112
4,800
...,...
10680,1478
10681,15000
10682,6000
10683,12000


# 5. Validação - Dividindo os dados em conjustos treinos e teste

In [18]:
from sklearn.model_selection import train_test_split  # função split do skeran

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_area_mobiliado_quartos, y_aluguel, test_size=0.2, random_state=42)

## 6. Criação do modelo

In [21]:
from sklearn.neighbors import KNeighborsRegressor # importando a função do modelo knn

In [23]:
knn = KNeighborsRegressor(n_neighbors=5) # criando o modelo de arquitetura do knn

In [24]:
knn.fit(x_train, y_train) # treinando o modelo, eu separei uma parte para treino ele esta pegando para treinar ela e fazer as regressãoo acertiva

KNeighborsRegressor()

# 7. Validação

In [25]:
from sklearn.metrics import mean_squared_error #  o comando from sklearn.metrics import mean_absolute_error é usado para importar a função mean_absolute_error da biblioteca scikit-learn,

In [26]:
from sklearn.metrics import mean_absolute_error

In [27]:
preditos = knn.predict(x_test) # # fazendo a predição, imagina que ele esta pegando o x da parte que separamos do teste e comparar com preditos

In [28]:
preditos # erray dos valores preditos

array([5140., 4553., 5178., ..., 1034., 1262., 1620.])

In [30]:
eqm = mean_squared_error(y_test, preditos) # lembra do y test que tem o valor dos laugueis agora vou comprar ele com o preditos para ver a taxa de erro square do meu modelo

In [31]:
eqm # erro quadrático médio

5864791.885465606

In [32]:
mae = mean_absolute_error(y_test, preditos) # erro médio absoluto

In [33]:
mae # erro médio absoluto

1583.8759943846514

# Interpretação dos Resultados

##Erro Quadrático Médio (EQM):

Valor: 5.864.791,89
Interpretação: O EQM é uma medida que penaliza erros grandes mais severamente do que erros pequenos, devido ao fato de que os erros são elevados ao quadrado. Um EQM de 5.864.791,89 indica que, em média, o quadrado das diferenças entre os valores previstos e os valores reais é bastante elevado, sugerindo que o modelo pode estar fazendo previsões com uma margem de erro significativa.



##Erro Absoluto Médio (MAE):

Valor: 1.583,88
Interpretação: O MAE representa a média das diferenças absolutas entre os valores previstos e os valores reais. Um MAE de 1.583,88 significa que, em média, as previsões do modelo estão a 1.583,88 unidades monetárias de distância dos valores reais. Este valor é mais intuitivo que o EQM, pois expressa o erro médio em unidades monetárias.


## Conclusão
O modelo apresenta uma margem de erro considerável, o que pode indicar a necessidade de ajustes adicionais no modelo ou nos dados. Isso pode incluir a seleção de novos atributos, ajuste de hiperparâmetros, ou a exploração de diferentes algoritmos de machine learning.
O EQM elevado sugere que existem alguns outliers ou previsões com erros significativos que estão influenciando o desempenho geral do modelo.
O MAE fornece uma medida mais direta do erro médio esperado, que pode ser útil para definir expectativas de precisão para os stakeholders.